In [ ]:
import pandas as pd

from google.colab import drive
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
drive.mount('/content/drive')

# Path file di Google Drive
file_path_1 = '/content/drive/MyDrive/capstone/diabetes_prediction_dataset.csv'
file_path_2 = '/content/drive/MyDrive/capstone/preprocessed_diabetes_data.csv'

# Membaca file CSV dari Google Drive
df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)


# Memeriksa apakah kedua dataset memiliki dimensi yang sama
if df1.shape == df2.shape:
    print("Dataset memiliki jumlah baris dan kolom yang sama.")

    # Membandingkan apakah kedua dataset sama persis
    if df1.equals(df2):
        print("Isi kedua dataset sama persis.")
    else:
        print("Isi kedua dataset berbeda.")
        # Menampilkan perbedaan secara lebih mendetail
        comparison = df1.compare(df2)
        print("Perbedaan antara kedua dataset:")
        print(comparison)
else:
    print("Dataset memiliki jumlah baris atau kolom yang berbeda.")

In [ ]:
drive.mount('/content/drive')

# Path file di Google Drive
file_path = '/content/drive/MyDrive/capstone/diabetes_prediction_dataset.csv'

# Membaca file CSV dari Google Drive
df = pd.read_csv(file_path)

In [ ]:
df.isna().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
def classify_diabetes(row):
    glucose = row['blood_glucose_level']
    hba1c = row['HbA1c_level']
    bmi = row['bmi']
    age = row['age']
    hypertension = row['hypertension']
    heart_disease = row['heart_disease']
    smoking = row['smoking_history']
    gender = row['gender']
    # Risiko berdasarkan smoking_history
    if glucose > 140 or hba1c >= 7.0 or (hypertension == 1 and heart_disease == 1):
        return 'Diabetes Tidak Terkendali'

    # Diabetes Terkendali
    if 126 <= glucose <= 140 or 6.5 <= hba1c < 7.0:
        if hypertension == 0 and heart_disease == 0:
            return 'Diabetes Terkendali'
        return 'Diabetes Tidak Terkendali'

    # Pra-diabetes
    if 100 <= glucose < 126 or 5.7 <= hba1c < 6.5 or bmi >= 25:
        # Gender-specific rules
        if gender == 'Female' and bmi >= 30:
            return 'Pra-diabetes'
        elif gender == 'Male' and bmi >= 27:
            return 'Pra-diabetes'
        # Additional risk factors
        if age > 45 or smoking in ['current', 'ever'] or hypertension == 1:
            return 'Pra-diabetes'

    # Normal
    return 'Normal'

# Menambahkan kolom diabetes_category dengan logika baru
df['diabetes_category'] = df.apply(classify_diabetes, axis=1)

print(df)

In [ ]:
label_encoder = LabelEncoder()
df['gender_encoded'] = label_encoder.fit_transform(df['gender'])

# Tentukan urutan yang diinginkan
smoking_order = ['No Info', 'never', 'former', 'current', 'not current', 'ever']

# Gunakan pd.Categorical untuk memberikan urutan pada kategori
df['smoking_history_encoded'] = pd.Categorical(df['smoking_history'], categories=smoking_order, ordered=True).codes

# Tentukan urutan yang diinginkan
diabetes_order = ['Normal', 'Pra-diabetes', 'Diabetes Terkendali', 'Diabetes Tidak Terkendali']

# Gunakan pd.Categorical untuk memberikan urutan pada kategori
df['diabetes_category_encoded'] = pd.Categorical(df['diabetes_category'], categories=diabetes_order, ordered=True).codes

print(df)

In [ ]:
# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=['gender','smoking_history','diabetes', 'diabetes_category', 'diabetes_category_encoded'])
y = df['diabetes_category_encoded']

print(X.head())

# Menentukan strategi sampling agar setiap kelas memiliki 9040 data
sampling_strategy = {
    3: 9040,  # Diabetes Tidak Terkendali
    2: 9040,  # Diabetes Terkendali
    1: 9040,  # Pra-diabetes
    0: 9040   # Normal
}

# Membuat instance RandomUnderSampler dengan sampling_strategy dictionary
under_sampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)

# Lakukan under-sampling pada dataset
X_resampled, y_resampled = under_sampler.fit_resample(X, y)

# Menampilkan distribusi kelas setelah under-sampling
print(f"Distribusi kelas setelah under-sampling: {Counter(y_resampled)}")

# Membagi data yang sudah di-resample menjadi data pelatihan dan data validasi
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Cek bentuk data untuk memastikan pembagian yang benar
print(f"Jumlah data pelatihan: {X_train.shape[0]}")
print(f"Jumlah data validasi: {X_val.shape[0]}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Membuat visualisasi distribusi kelas setelah under-sampling
plt.figure(figsize=(10, 6))  # Ukuran gambar lebih besar agar label lebih leluasa
ax = sns.countplot(x=y_resampled, palette='viridis')

# Menambahkan jumlah angka di dalam setiap batang dengan sedikit jarak dari batang
for p in ax.patches:
    # Mengambil tinggi (jumlah) batang
    height = p.get_height()
    # Menambahkan angka di dalam batang dengan sedikit jarak (misalnya +200)
    ax.text(p.get_x() + p.get_width() / 2, height / 2 + 200, f'{int(height)}',
            ha='center', va='center', fontsize=12, color='white')

# Menambahkan judul dan label
plt.title("Distribusi Kelas Setelah Under-Sampling")
plt.xlabel("Kelas Diabetes")
plt.ylabel("Jumlah Data")

# Mengatur label sumbu x dengan jarak yang lebih lebar agar tidak tumpang tindih
plt.xticks(ticks=[0, 1, 2, 3],
           labels=['Normal', 'Pra-diabetes', 'Diabetes Terkendali', 'Diabetes Tidak Terkendali'],
           rotation=0,  # Tidak ada pemiringan
           ha='center')  # Menjaga label tetap lurus dan terpusat

# Menampilkan plot
plt.show()


In [ ]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(8,)),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Definisikan early stopping untuk menghindari overfitting
early_stopping = EarlyStopping(monitor='val_loss',  # Memantau 'val_loss' untuk menghentikan pelatihan lebih awal
                               patience=10,         # Berhenti jika tidak ada peningkatan dalam 10 epoch
                               restore_best_weights=True)

# Melatih model dengan data pelatihan dan validasi
history = model.fit(X_train, y_train, epochs=80, batch_size=32,  # Atur epoch sesuai kebutuhan
          validation_data=(X_val, y_val), callbacks=[early_stopping])  # Menggunakan data validasi

In [ ]:
# Menghitung akurasi pada data test
test_loss, test_accuracy = model.evaluate(X_val, y_val)

# Menampilkan akurasi dalam persen
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt

# Plot Loss Curve
plt.figure(figsize=(12, 6))

# Plot loss pelatihan dan validasi
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot Accuracy Curve
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Menampilkan plot
plt.tight_layout()
plt.show()


In [ ]:
model.save('diabetes_model.keras')